In [1]:
# installiamo le librerie necessarie
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain_openai
!pip install --upgrade --quiet langchain_chroma
!pip install --upgrade --quiet langchain_community

In [2]:
# Carichiamo l'API Key di OpenAI
import os
os.environ['OPENAI_API_KEY'] = 'xxx'

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# Usiamo il Document Loader per caricare il video
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./i_promessi_sposi.txt")
documents = loader.load()

# Il nostro documento è troppo grande per entrare nella context windows di GPT
# Usiamo un text spliltter per dividerlo in documenti più piccoli
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=5000,
)
documents = text_splitter.split_documents(documents)
print(f"Diviso il testo in {len(documents)} documenti")

Diviso il testo in 271 documenti


In [4]:
# creiamo un vectorstore con i nostri documenti
print("Creiamo il vectorstore con i documenti...")
db = Chroma.from_documents(documents, OpenAIEmbeddings())
print("Creato il vectorstore con i documenti")

# Per rispondere ad una domanda dobbiamo trovare i documenti più rilevanti. Lo faremo cercando la similarità tra il testo della domanda e i documenti
retriever = VectorStoreRetriever(vectorstore=db, search_type="similarity", search_kwargs={"k": 10})

# preparamo un prompt che usi la domanda e i documenti
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", """
Utilizza il contenuto del testo fornito per rispondere alla domanda dell'utente. Rispondi che non conosci la risposta se non è contenuta nel testo.
Domanda: {question}
	 
Testo:
{text}
""")
])

# Carichiamo il modello di GPT
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# componiamo la chain
chain = ({"text": RunnablePassthrough(), "question": RunnablePassthrough()} | prompt_template | llm)

Creiamo il vectorstore con i documenti...
Creato il vectorstore con i documenti


In [5]:
# Inserisci una domanda relativa al contenuto del testo
question = "Chi era Don Abbondio?"

# troviamo i documenti simili
similar_documents = retriever.invoke(question)
print(f"Trovati {len(similar_documents)} documenti simili")

# formattiamo i documenti
text = ""
for document in similar_documents:
	text += document.page_content + "\n"
	
# Eseguiamo la chain
answer = chain.invoke({"text": text, "question": question})

# Stampiamo il risultato
print(answer.content)

Trovati 10 documenti simili
Don Abbondio era un curato, un uomo di chiesa, che non era nato con un cuore coraggioso. Fin dai suoi primi anni, aveva compreso che la peggior condizione, a quei tempi, era quella di un individuo senza mezzi di difesa, e che non si sentiva inclinato a essere divorato. La forza legale non proteggeva l'uomo tranquillo e inoffensivo, e don Abbondio aveva quindi sviluppato un sistema per evitare i contrasti e cedere in quelli che non poteva evitare. Era un uomo che cercava di mantenere la propria quiete, evitando di prendere parte a conflitti e cercando di stare dalla parte del più forte quando costretto a scegliere. Era anche un rigido censore degli uomini che non si comportavano come lui, ma solo quando poteva farlo senza pericolo.
